# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [6]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


# 1. What test is appropriate for this problem? Does CLT apply?

We are ultimately interested in comparing the difference between the proportion of callbacks from two different groups. Utlimately, our null hypothesis would look something like $H_{0}: p_{b} = p_{w}$ where we would be interested in seeing if the proportion of callbacks received is dependant on whether or not an applicant's name sounds "black or white".

In [11]:
w = data[data.race=='w']
b = data[data.race=='b']
num_w = len(w)
num_b = len(b)
print("The number of applications with a white sounding name : " + str(num_w))
print("The number of applications with a black sounding name : " + str(num_b))

The number of applications with a white sounding name : 2435
The number of applications with a black sounding name : 2435


In [14]:
w_call = w[w.call == 1]
b_call = b[b.call == 1]
wc = len(w_call)
bc = len(b_call)
print('The number of white sounding names that got callbacks : ' + str(wc))
print('The number of black sounding names that got callbacks : ' + str(bc))

The number of white sounding names that got callbacks : 235
The number of black sounding names that got callbacks : 157


In [15]:
p_w =  wc/num_w
p_b =  bc/num_b
print('The proportion of white sounding names to get a callback : ' + str(p_w))
print('The proportion of black sounding names to get a callback : ' + str(p_b))

The proportion of white sounding names to get a callback : 0.09650924024640657
The proportion of black sounding names to get a callback : 0.06447638603696099


To check if this can be modelled as a normal distribution, we must check to see if $np \geq 10$ and $n(1-p) \geq 10$

In [17]:
print(num_b * p_b)
print(num_b * (1-p_b))
print("---")
print(num_w * p_w)
print(num_w * (1-p_w))

157.0
2278.0
---
235.0
2200.0


Since we know that this can be modelled as a normal distribution, we can now find the standard error, and construct a confidence interval and margine of error using a designated z-score of 1.96 (95% confidence)

# 2. What are the null and alternate hypotheses?

**Null Hypothesis** $H_{0} : p_{w} = p_{b}$ <br>
**Allternate Hypothesis** $H_{\alpha} : p_{w} \neq p_{b}$

# 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

**Boostrap Approach**


In [30]:
def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1, data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

In [31]:
def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1,data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates

In [32]:
def diff_of_means(data_1, data_2):
    """Difference in means of two arrays."""

    # The difference of means of data_1, data_2: diff
    diff = np.mean(data_1) - np.mean(data_2)

    return diff

The dataframs w_boot and b_boot below contian only the callback responses (1 or 0). Since they are a simple array of either 1 or 0, the mean of the array is the overall callback rate. We will use this callback rate, and generate permutation replicates to see if the empirical difference in means of white sounding names and black sounding names is plausible. 

In [33]:
w_boot = w.call
b_boot = b.call

In [38]:
# Compute difference of mean callback rate: empirical_diff_means
empirical_diff_means = np.mean(w_boot) - np.mean(b_boot)
empirical_diff_means

0.03203285485506058

In [36]:
# Draw 10,000 permutation replicates: perm_replicates
perm_replicates = draw_perm_reps(w_boot,b_boot,
                                 diff_of_means, size=10000)

In [37]:
# Compute p-value: p
p = np.sum(perm_replicates >= empirical_diff_means) / len(perm_replicates)
# Print the result
print('p-value =', p)

p-value = 0.0


The p-value above shows the probability of getting at least 0.032 difference in mean of callback rates between white sounding names and black sounding names. Since the p value is 0, and is lower than the threshold of $\alpha = 0.05$ we reject the null hypothesis that the difference in callback rates (difference in means in this case) is equal to zero. <br>
That is, it is not sound that we see this empirical difference in means in the first place. 

**Frequentist statistical approach**

In [20]:
z = 1.96
margin = z * np.sqrt( ( p_w*(1-p_w) / num_b) + (p_b*(1-p_b)/num_w) )

print("Margin of error = ", margin)

Margin of error =  0.015255406349886438


In [22]:
print("The confidence interval is given by :", p_w-p_b-z*margin,"to", p_w-p_b+z*margin)

The confidence interval is given by : 0.0021322577636681654 to 0.061933450655223


We can see in the confidence interval above that 0 is not included. Thus, we would not expect the two proportions of callbacks between the races to equal each other. 

In [25]:
from statsmodels.stats.proportion import proportions_ztest as pz
z_result = pz(np.array([bc,wc]),np.array([num_b,num_w]),value=0)

In [27]:
print('The p value obtained for testing the null hypothesis is : ' + str(z_result[1]))

The p value obtained for testing the null hypothesis is : 3.983886837585077e-05


# 4. Write a story describing the statistical significance in the context or the original problem.


Since the p value is extremely small, and less than the selected threshold of $\alpha = 0.05$ we reject the null hypothesis. <br>
We find that the subjective field of "white sounding names" and "black sounding names" **DOES** have a significant influence on the outcome of job applicants getting a callback. <br>
By suggesting that these racially charged names were applied randomly to the sample dataset above, we find that that there is indeed some sort of racial discrimination in the US Job Market in terms of applicants getting a callback.

# 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

This analysis does not mean that race is the most important faactor in callback success. It only shows that race does play a factor in determining whether an applicant recieves a callback or not. We would need to replicate the process for many of the other factors to see which factor has the most influence in callback success.